<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Implementación de un Algoritmo Genético para Resolver el Problema del Bipedal Walker con Gym<a id="top"></a>

<i><small>Grupo: NeoTech<br>Última actualización: 2023-05-20</small></i></div>


***

## Introducción y objetivos

El problema del **Bipedal Walker** representa un reto en el campo de la inteligencia artificial donde el objetivo es desarrollar un algoritmo capaz de controlar el movimiento de un andador bípedo. 
 
Este proyecto propone  un enfoque basado en algoritmos genéticos para resolver este reto. Los algoritmos genéticos son técnicas inspiradas en la evolución biológica que combinan selección, cruce y mutación para encontrar soluciones óptimas en un espacio de búsqueda. En este contexto, se utiliza un **algoritmo genético con Gym** para proporcionar un entorno  simulado para evaluar el rendimiento de un Bipedal Walker.

## Imports y configuración

A continuación importaremos las librerías que se usarán a lo largo del _notebook_. Se deberá tambien instalar Anaconda.

In [ ]:
import gym
import random
import numpy as np
from operator import attrgetter
import matplotlib.pyplot as plt

***

## Definición de las clases

- **Población**: Representa una población de individuos en el algoritmo genético
- **Individuo**: Representa a un individuo en la población

## Métodos de la clase Individuo

(Hace falta llamar a env antes)

In [ ]:
class Individuo:
    # Inicializa el individuo con una lista de acciones aleatorias
    def __init__(self, num_acciones):
        self.acciones = [env.action_space.sample() for _ in range(num_acciones)]
        self.valor_fitness = 0
        self.probabilidad_seleccion = 0

    # Calcula el valor de fitness del individuo en base a la recompensa total obtenida
    def calcular_valor_fitness(self, recompensa_total):
        self.valor_fitness = recompensa_total

    # Establece la probabilidad de selección del individuo en la población
    def set_probabilidad_seleccion(self, probabilidad):
        self.probabilidad_seleccion = probabilidad

## Métodos de la clase Poblacion

In [ ]:
class Poblacion:
    def __init__(self):
        self.individuos = []

    # Calcula la probabilidad de selección de cada individuo en la población 
    # basándose en su valor de fitness
    def calcular_probabilidad_seleccion(self):
        recompensas_normalizadas = []
        recompensas_minimas = float("inf")
        suma_recompensas = 0

        for individuo in self.individuos:
            if individuo.valor_fitness <= recompensas_minimas:
                recompensas_minimas = individuo.valor_fitness

        for individuo in self.individuos:
            suma_recompensas += individuo.valor_fitness + abs(recompensas_minimas) + 1

        for individuo in self.individuos:
            recompensas_normalizadas.append((individuo.valor_fitness + abs(recompensas_minimas) + 1) / suma_recompensas)

        # Creamos la distribución de probabilidad de selección
        for i in range(len(recompensas_normalizadas)):
            self.individuos[i].set_probabilidad_seleccion(recompensas_normalizadas[i])

    # Selecciona aleatoriamente dos Individuos de la población para el cruce
    def seleccionar_pares_individuos(self):
        probabilidades_acumulativas = []
        suma_parcial = 0

        for individuo in self.individuos:
            suma_parcial += individuo.probabilidad_seleccion
            probabilidades_acumulativas.append(suma_parcial)

        numero_aleatorio_1 = random.uniform(0.0, 1.0)
        numero_aleatorio_2 = random.uniform(0.0, 1.0)
        indice_seleccionado_1 = -1
        indice_seleccionado_2 = -1

        # Obtenemos los índices de los individuos seleccionados
        for i in range(len(probabilidades_acumulativas)):
            if numero_aleatorio_1 <= probabilidades_acumulativas[i]:
                indice_seleccionado_1 = i
                break

        for i in range(len(probabilidades_acumulativas)):
            if numero_aleatorio_2 <= probabilidades_acumulativas[i]:
                indice_seleccionado_2 = i
                break

        return indice_seleccionado_1, indice_seleccionado_2

    # Ejecuta el entorno de Gym para cada individuo de la población durante un número determinado de iteraciones (playout)
    # y calcula el valor de fitness de cada individuo en base a las recompensas obtenidas
    def realizar_playout(self, num_individuos, num_acciones, num_playout, env):
        for i in range(num_individuos):
            env.reset()
            suma_recompensas = 0

            for _ in range(num_playout // num_acciones):
                for j in range(num_acciones):
                    accion = self.individuos[i].acciones[j]
                    _, recompensa, _, _ = env.step(accion)
                    suma_recompensas += recompensa

            self.individuos[i].calcular_valor_fitness(suma_recompensas)

    # Realiza la mutación en los individuos de la población con una cierta probabilidad y número de mutaciones
    def mutacion(self, num_acciones, porcentaje_mutacion, num_mutaciones):
        for individuo in self.individuos:
            probabilidad_mutacion = random.randint(0, 100)

            if probabilidad_mutacion >= porcentaje_mutacion * 100:
                continue

            # Generar los índices de acciones a mutar
            indices_mutacion = random.sample(range(num_acciones), num_mutaciones)

            for indice in indices_mutacion:
                individuo.acciones[indice] = env.action_space.sample()

    # Realiza el cruce entre los individuos de la población para generar una nueva población
    def cruce(self, num_acciones, elite):
        nueva_poblacion = []
        nueva_poblacion.extend(elite)

        for i in range(len(elite), len(self.individ.uos), 2):
            padre1, padre2 = self.individuos[i], self.individuos[i + 1]
            hijo1 = Individuo(num_acciones)
            hijo2 = Individuo(num_acciones)
            punto_corte = random.randint(1, num_acciones - 1)

            hijo1.acciones = padre1.acciones[:punto_corte] + padre2.acciones[punto_corte:]
            hijo2.acciones = padre2.acciones[:punto_corte] + padre1.acciones[punto_corte:]

            nueva_poblacion.extend([hijo1, hijo2])
        self.individuos = nueva_poblacion

    # Ejecuta el algoritmo genético durante un número determinado de generaciones, utilizando los métodos de antes
    def ejecutar_algoritmo_genetico(self, num_generaciones, num_individuos, num_acciones, num_playout, porcentaje_mutacion, num_mutaciones, env):
        mejores_fitness = []

        for _ in range(num_generaciones):
            elite = self.seleccionar_elite(num_individuos)
            self.calcular_probabilidad_seleccion()
            self.realizar_playout(num_individuos, num_acciones, num_playout, env)
            self.mutacion(num_acciones, porcentaje_mutacion, num_mutaciones)
            self.cruce(num_acciones, elite)

            mejor_individuo = max(self.individuos, key=attrgetter('valor_fitness'))
            mejores_fitness.append(mejor_individuo.valor_fitness)

        return mejores_fitness


***

## Configuración de parámetros

In [ ]:
num_generaciones = 100           # Número de generaciones
num_individuos = 50              # Número PAR de individuos en la población
num_acciones = 40                # Número de acciones posibles por individuo (múltiplo de 4 debido a la mutacion -0.75)
num_playout = 500                # Número de iteraciones (playout) por individuo
porcentaje_mutacion = 0.5        # Porcentaje de individuos que sufrirán mutación
num_mutaciones = 1               # Número de mutaciones que se aplicarán por individuo

## Crear el ambiente de Gym

In [ ]:
env = gym.make('BipedalWalker-v3')

## Crear la población inicial

In [ ]:
poblacion = Poblacion()
for _ in range(num_individuos):
    individuo = Individuo(num_acciones)
    poblacion.individuos.append(individuo)

## Ejecutar el algoritmo genético

In [ ]:
mejores_fitness = poblacion.ejecutar_algoritmo_genetico(num_generaciones, num_individuos, num_acciones, num_playout, porcentaje_mutacion, num_mutaciones, env)

***

## Mostrar resultados

In [ ]:
plt.plot(range(num_generaciones), mejores_fitness)
plt.xlabel('Generación')
plt.ylabel('Mejor Fitness')
plt.title('Evolución del Mejor Fitness')
plt.show()

//METER FOTOS